# Information Retrieval Practice

In this practice we are going to build our basic information retrieval model. 

To that end, the first step is to create the inverted index. In particular, we will create a python dictionary with word as key and a dictionary as value. This nested dictionary has a document where the word occurs as key and log term frequency(tf) value as value.

Once created this index, for a new user query we compute the result set: documents including the words in the user query.

For the term weighting (in the inverted index creation and the query processing), we propose two methodologies:
 - Binary: this performs unranked search, producing all documents which satisfy the query. The query can have one or more words.
 - TF-IDF: The result-set is another python dictionary with the document index as key and the product of the stored log term-frequency and the calculated-on-the-fly-inverted-document-frequency as the value.

Then the result-set is reverse-sorted based on it values and the top 10 documents are displayed.


For the practice, I provides you a set of function that you might find useful

In [20]:
from nltk.corpus import stopwords
import re

stopwords_english = set(stopwords.words('english'))

def get_list_tokens_nltk(corpus, file_name):
    string = get_raw_text(corpus,file_name)
    return get_list_tokens_string(string)


def get_list_tokens_string(string):
    list_words = re.split(r'\W+',string)
    return [w.lower() for w in list_words if w.isalpha() and len(w)>1 and w.lower() not in stopwords_english]    


def get_raw_text(corpus,file_name):
    string=''
    if corpus=='mr':
        from nltk.corpus import movie_reviews
        string = movie_reviews.raw(fileids=file_name)
    else:
        from nltk.corpus import reuters
        string = reuters.raw(fileids=file_name)
    return string

def get_list_fileids(corpus):
    li=[]
    if corpus=='mr':
        from nltk.corpus import movie_reviews
        li = movie_reviews.fileids()
    else:
        from nltk.corpus import reuters
        li = reuters.fileids()
    return li


Needed imports

In [21]:
from __future__ import division
import time
from math import log

As a corpus to query for information we use the Reuters Dataset:

https://archive.ics.uci.edu/ml/datasets/reuters-21578+text+categorization+collection

It is included among the NLTK corpus.

In [22]:
corpus='reuters'

# Exercise #1 - Binary Score

Let's start with the binary weighting

## Exercise 1.1: Index Generation

This first exercise is focused on creating the inverted index by using binary weighting.

To that end we:
 1. Go over all the documents (ids and text) in the corpus
 2. Get the tokens related to the text in the document
 3. Update the inverted index with each token and the id of the document


In [23]:
start_time = time.time()

# Get the list if documents ids in the corpus
list_document_ids =  get_list_fileids(corpus)

print "List of documents ids", list_document_ids[:5]
print ""

## 1) Create a inverted index: dictionary with word as key and list of documents where it occurs in sorted order as value
inverted_index = {}

## 2)Loop through the dataset, to get the entire text from  each file
for (document_index, file_name) in enumerate(list_document_ids):
    
    ##3) Parse the string to get individual words    
    list_words = get_list_tokens_nltk(corpus, file_name)

    ## 4) Update the dictionary with the words in the document and the related document_id
    for w in set(list_words):
        if inverted_index.get(w,0) == 0: # Check if w is not in the vocabulary
            inverted_index[w]=[]
           
        inverted_index[w].append(document_index)

print "Inverted Index has been prepared and it took"
print time.time() - start_time, "seconds"

print ""
print "Example of List of words for document", document_index, list_words[:10]
print ""

print ""
print "Example of entry for word automotive", inverted_index['automotive']
print ""

List of documents ids ['test/14826', 'test/14828', 'test/14829', 'test/14832', 'test/14833']

Inverted Index has been prepared and it took
2.91588711739 seconds

Example of List of words for document 10787 [u'lt', u'automotive', u'technologies', u'corp', u'year', u'net', u'shr', u'cts', u'vs', u'cts']


Example of entry for word automotive [310, 544, 963, 1260, 1600, 2194, 3548, 3700, 3932, 3992, 4065, 4066, 4447, 5489, 5836, 6323, 6453, 6597, 6703, 8975, 9369, 9440, 9445, 10260, 10281, 10628, 10637, 10714, 10787]



## Exercise 1.2: Getting the query from the user

Once created the index, we can answer to user queries. The strategy is to:
 1. Get the tokens in the user query
 2. Get the documents related to the queries (by using the inverted_index). Herein we allow OR and AND queries:
 
 2.1 OR: We only need to aggreagate all the documents in which the tokens in the query appear (tokens in the query are the keys of the inverted_index dictionary and the values are the related documents)
         
 2.2. AND: We get only the documents in which ALL the tokens in the query appear.

#### Experiment with different queries and change the operator (OR, AND) to see how the results are affected

In [24]:
query = raw_input("Enter your query string : ")
op = raw_input("Enter the operator, (AND/OR) Default is AND: ")

start_time = time.time()
query_list = get_list_tokens_string(query)
result_file_indices= []

if op=='OR' or op=='or': # OR query
    for q in query_list:
        print "Retrieving documents including", q
        print ""
        if inverted_index.get(q.lower(), 0)!=0: # If there is any document including q
            print "Documents including", q, ":", inverted_index[q.lower()]
            print ""
            result_file_indices.extend(inverted_index[q.lower()])
            
else: # AND query
    file_indices=range(len(list_document_ids))
    for q in query_list:
        print "Retrieving documents including", q
        print ""
        if inverted_index.get(q.lower(), 0)==0: # Check if the term is included in the inverted index
            print "Term", q, "does not appear in the corpus"
            print ""
            file_indices=[] #Return empty document list       
            break
        else:
            temp_list=[]
            query_file_indices = inverted_index[q.lower()]
            print ""
            index_f=0
            index_q=0
            # Go over the document list and the result list (documents including q)
            print "Documents including previous query terms:", file_indices
            print ""
            print "Documents including", q, ":", query_file_indices
            print ""
            while index_f < len(file_indices) and index_q < len(query_file_indices):
                if file_indices[index_f] == query_file_indices[index_q]:
                    # Add to the final result list only those documents included in both lists
                    temp_list.append(query_file_indices[index_q])
                    index_f+=1
                    index_q+=1
                elif file_indices[index_f] < query_file_indices[index_q]:
                    index_f+=1
                else:
                    index_q+=1
            file_indices=[]
            file_indices.extend(temp_list)
            print ""
            print ""
    result_file_indices.extend(file_indices)

if(len(result_file_indices)==0):
    print "Sorry No matches found"
else:
    print "Number of search results : " , len(result_file_indices)
    print "Results:"
    print "\n"
    for index in result_file_indices:
        print get_raw_text(corpus, list_document_ids[index])
        print "\n"

Enter your query string : bank crisis
Enter the operator, (AND/OR) Default is AND: OR
Retrieving documents including bank

Documents including bank : [4, 9, 11, 12, 17, 23, 31, 32, 33, 43, 45, 46, 67, 70, 71, 80, 83, 114, 115, 118, 130, 132, 191, 195, 204, 269, 271, 279, 285, 286, 290, 302, 308, 317, 319, 322, 323, 326, 328, 330, 331, 332, 336, 339, 358, 361, 362, 363, 365, 367, 374, 377, 379, 382, 383, 384, 385, 390, 391, 402, 427, 431, 437, 487, 496, 497, 506, 508, 510, 512, 522, 526, 531, 539, 544, 545, 625, 626, 643, 649, 663, 680, 683, 684, 686, 687, 689, 690, 691, 696, 709, 713, 717, 718, 719, 725, 732, 739, 749, 755, 759, 760, 761, 764, 767, 776, 781, 784, 798, 799, 811, 815, 817, 821, 824, 829, 830, 838, 839, 841, 842, 848, 853, 868, 876, 892, 898, 907, 908, 918, 920, 950, 953, 954, 963, 985, 1004, 1016, 1018, 1029, 1031, 1034, 1040, 1046, 1069, 1086, 1087, 1096, 1097, 1099, 1105, 1111, 1120, 1121, 1127, 1133, 1156, 1157, 1158, 1174, 1187, 1195, 1197, 1201, 1202, 1208, 1209, 12

# Exercise 2 - TF-IDF Score

In the previous exercise, we used a simple binary weighting for the terms (i.e., whether the term appears or not in the documents). Although this approach might be enough for small documents (e.g, tweets) it is not able to properly capture the importance of the terms in larger documents.

In this exercise, we will apply TF-IDF as a weighting score.
        
       

## Exercise 2.1: Index Generation

Change the inverted index generation to use tf-idf weighting

In [25]:
start_time = time.time()    
list_document_ids =  get_list_fileids(corpus)   
num_docs = len(list_document_ids)    

##1) Create a inverted index
inverted_index = {}

##2)Loop through the dataset, to get the entire text from  each file
for (document_index, file_name) in enumerate(list_document_ids):
    
    ##3) Parse the string to get individual words    
    list_words = get_list_tokens_nltk(corpus, file_name)
    
    ##4) Update the dictionary to use tf-idf 
    for w in list_words:
        if inverted_index.get(w,0)==0:
            inverted_index[w]={}
            inverted_index[w][document_index]=1
        else:
            inverted_index[w][document_index] = inverted_index[w].get(document_index,0)
            inverted_index[w][document_index] += 1
    for w in set(list_words):
        inverted_index[w][document_index]=1+log(inverted_index[w][document_index])

        
print "Inverted Index has been prepared and it took"
print time.time() - start_time, "seconds"


Inverted Index has been prepared and it took
3.58062982559 seconds


In [26]:
print ""
print "Example of entry for word automotive", inverted_index['automotive']
print ""


Example of entry for word automotive {10628: 1.6931471805599454, 10637: 1.6931471805599454, 8975: 1.0, 2194: 1.0, 10260: 1.0, 3992: 1.0, 9369: 1.0, 544: 1.0, 10787: 1.0, 3548: 1.0, 6703: 1.0, 6323: 1.0, 6453: 1.0, 310: 2.09861228866811, 1600: 1.0, 963: 1.0, 6597: 1.0, 5836: 1.0, 10714: 1.0, 3932: 1.6931471805599454, 4447: 1.0, 9440: 1.0, 4065: 1.0, 4066: 1.0, 9445: 1.0, 1260: 1.0, 5489: 1.6931471805599454, 3700: 1.0, 10281: 1.0}



## Exercise 2.2: User query

Change the weighting of the terms in the query to use tf-idf

In [27]:
##5) Getting the query from the user
query = raw_input("Enter your query string : ")
result_file_dict={}

start_time = time.time()

##6) Tokenizing query string to get individual words
query_list = get_list_tokens_string(query)

##7) Calculating tf-idf scores 
for q in query_list:
    d = inverted_index.get(q,0) 
    if d!=0:
        length=len(d)
        for file_index in d.keys():
            result_file_dict[file_index] = result_file_dict.get(file_index,0)
            result_file_dict[file_index]+=((1+log(d[file_index]))*(log(num_docs/length)/log(10)))
                                        #1st term is tf # 2nd term is idf

##8) Sorting the dictionary based on its values            
result_file_indices = sorted(result_file_dict.items(), key=lambda x:x[1],reverse = True)

print "Time taken to search"
print (time.time() - start_time), "seconds"

if(len(result_file_indices)==0):
    print "Sorry No matches found"
else:
    print "Number of search results : " , len(result_file_indices)
    print "Results:"
    print "\n"
    if len(result_file_indices) > 10:
        print "Returning 1st 10 results"
    for (index,tup) in enumerate(result_file_indices):
        if index==10:
            break
        print get_raw_text(corpus, list_document_ids[tup[0]])
        print "\n"



Enter your query string : bank crisis
Time taken to search
0.0153679847717 seconds
Number of search results :  1396
Results:


Returning 1st 10 results
CONABLE WARNS PROTECTIONISM MIGHT SPREAD
  World Bank President Barber Conable
  expressed concern that trade protectionism, at the heart of a
  new showdown between the United States and Japan, might spread
  throughout the industrial world.
      But in an interview with Reuters, Conable said the action
  by the United States to slap tariffs on certain electronic
  goods from Japan did not mean the countries were heading for a
  full-scale trade war.
      Conable said the World Bank has been pressing developing
  countries to open their markets, arguing that a free trading
  environment increased the possibility of global economic
  growth.
      "We have, in fact, been making adjustment loans to many
  countries in the developing world which have encouraged the
  opening of their markets and we want to be sure that the
  developed w

# Exercise 3: Improve term tokenization

We have only applied a basic tokenization approach and stopword removal to deal with the terms in the text. But we can do better. As seen in class there are different procedures that can improve the operation of the information retrieval system:
 - Stemming
 - Phrase queries
 
Let's try to include them in our model

## Exercise 3.1: Stemming

Modify the `get_list_tokens_string` function to stem the tokens in the text. To that end, NLTK provides different stemming algorithms in the `nltk.stem` package

In [28]:
from nltk.stem import *

def get_list_tokens_string(string):
    list_words = re.split(r'\W+',string)
    list_words = [w.lower() for w in list_words if w.isalpha() and len(w)>1 and w.lower() not in stopwords_english]  
    # Include here the code for stemming
    stemmer = PorterStemmer()
    list_words = [stemmer.stem(w) for w in list_words]
    return list_words

Executing again the code with the stemming

In [29]:
start_time = time.time()    
list_document_ids =  get_list_fileids(corpus)   
num_docs = len(list_document_ids)    

##1) Create a inverted index
inverted_index = {}

##2)Loop through the dataset, to get the entire text from  each file
for (document_index, file_name) in enumerate(list_document_ids):
    
    ##3) Parse the string to get individual words    
    list_words = get_list_tokens_nltk(corpus, file_name)
    
    ##4) Update the dictionary to use tf-idf 
    for w in list_words:
        if inverted_index.get(w,0)==0:
            inverted_index[w]={}
            inverted_index[w][document_index]=1
        else:
            inverted_index[w][document_index] = inverted_index[w].get(document_index,0)
            inverted_index[w][document_index] += 1
    for w in set(list_words):
        inverted_index[w][document_index]=1+log(inverted_index[w][document_index])

        
print "Inverted Index has been prepared and it took"
print time.time() - start_time, "seconds"

##5) Getting the query from the user
query = raw_input("Enter your query string : ")
result_file_dict={}

start_time = time.time()

##6) Tokenizing query string to get individual words
query_list = get_list_tokens_string(query)

##7) Calculating tf-idf scores 
for q in query_list:
    d = inverted_index.get(q,0) 
    if d!=0:
        length=len(d)
        for file_index in d.keys():
            result_file_dict[file_index] = result_file_dict.get(file_index,0)
            result_file_dict[file_index]+=((1+log(d[file_index]))*(log(num_docs/length)/log(10)))
                                        #1st term is tf # 2nd term is idf

##8) Sorting the dictionary based on its values            
result_file_indices = sorted(result_file_dict.items(), key=lambda x:x[1],reverse = True)

print "Time taken to search"
print (time.time() - start_time), "seconds"

if(len(result_file_indices)==0):
    print "Sorry No matches found"
else:
    print "Number of search results : " , len(result_file_indices)
    print "Results:"
    print "\n"
    if len(result_file_indices) > 10:
        print "Returning 1st 10 results"
    for (index,tup) in enumerate(result_file_indices):
        if index==10:
            break
        print get_raw_text(corpus, list_document_ids[tup[0]])
        print "\n"

Inverted Index has been prepared and it took
23.9896919727 seconds
Enter your query string : bankers
Time taken to search
0.00171494483948 seconds
Number of search results :  178
Results:


Returning 1st 10 results
ECONOMIC SPOTLIGHT - KUWAITI ECONOMY
  Kuwait's oil-reliant and debt-ridden
  economy has started to pull out of a nosedive but oil prices
  will determine the pace of recovery, bankers and economists
  say.
      Crucial will be the ability of the 13-member OPEC to hold
  oil prices around a new benchmark of 18 dlrs a barrel in the
  northern hemisphere summer when demand usually slackens.
      Bankers estimate the economy, measured in terms of gross
  domestic product (gdp), shrank 19 pct in real terms last year
  after contracting 8.1 pct the year before.
      This was after taking into account inflation in consumer
  prices of 1.5 pct in 1985, slowing to 1.0 pct in 1986.
      Factors depressing economic activity include the
  6-1/2-year-old Iran-Iraq war on Kuwait's d

Check the improvements achieved by means of the stemming in the IR system. What does it retrieve for queries such as `bankers`?

## Exercise 3.2: Phrase queries

Now I know I said that our inverted index would map words to document names, but, we also want to support phrase queries: queries for not only words, but words in a specific sequence. For this, we’ll need to know where in each document each word shows up, so we can check for order. I used each word’s index in the tokenized list for a document as the position of the word in that document.

For simplicity, we are going to take as starting point the binary weighting approach.

So our first task, then, is to create a mapping of words to their positions for each document, and then combine these to create our complete inverted index.

The following function takes a list of tokens and returns a dictionary with the token and the position/s of the token in the document

In [12]:
#input = [word1, word2, ...]
#output = {word1: [pos1, pos2], word2: [pos2, pos434], ...}
def index_one_file(termlist):
	fileIndex = {}
	for index, word in enumerate(termlist):
		if word in fileIndex.keys():
			fileIndex[word].append(index)
		else:
			fileIndex[word] = [index]
	return fileIndex

By using the previous function, we can now update our index to store for each token not only the document/s in which it appears, but also the positions

In [13]:
start_time = time.time()    
list_document_ids =  get_list_fileids(corpus)   
num_docs = len(list_document_ids)    

##1) Create a inverted index
inverted_index = {}

## 2)Loop through the dataset, to get the entire text from  each file
for (document_index, file_name) in enumerate(list_document_ids):
    list_words = get_list_tokens_nltk(corpus, file_name)
    list_words_pos = index_one_file(list_words)
    
    ## 3) Update the dictionary with the words in the document and the related document_id
    for w in set(list_words):
        if w in inverted_index.keys():
            if file_name in inverted_index[w].keys():
                inverted_index[w][file_name].extend(list_words_pos[w][:])
            else:
                inverted_index[w][file_name] = list_words_pos[w]
        else:
            inverted_index[w] = {file_name: list_words_pos[w]}


Taking a look to the inverted index

In [11]:
inverted_index.items()[:5]

[(u'xylen', {'test/19903': [79, 81]}),
 (u'hallwood', {'training/6325': [0]}),
 (u'petrolan', {'test/20515': [0, 60], 'test/20669': [0, 48]}),
 (u'surpasss', {'test/15939': [38]}),
 (u'middleman', {'training/1421': [74, 105]})]

We also have to adapt the query proces to allow the phrase queries.

**Intermediate results**
Then, we run a single word query for every word in the input, and add each of these result lists to our total list. Then, we create a set called setted, which takes the intersection of the first list with all the other list (essentially taking the intersection of all of the lists), and leaves us with our intermediate result set: all the documents that contain all the words in the query.

**Ordering**
Then, we have to check for ordering. So, for every list in the intermediate results, we first make a list of lists of the positions of each word in the input query. Then (pay attention here!) we use two nested for loops to iterate through this list of lists. For every position in every list, we subtract a number i, which increases as we go through the list of lists. i increments by 1 when we got through the list of lists. Now, remember that python lists preserve order, so this list of lists contains the position lists of every word in the original query in the order of the words in the original query. Then, if these words are in the proper order, and we subtract an integer, i, from every position in each position list, and i increments by 1 as we go to each successive position list, then, if these phrases are in order, the intersection of all of these modified lists of lists must have a length of at least one.

Let me give an example:

Lets say the phrase we’re querying for is “the cake is a lie.” Then, for a specific filename, assume these are the positions of each word:

    the : [23, 34, 15]
    cake: [19, 35, 12]
    is: [179, 36, 197]
    a: [221, 37, 912]
    lie: [188, 6, 38]
Then, our list of lists is:

    [[23, 34, 15], [19, 35, 12], [179, 36, 197], [221, 37, 912], [188, 6, 38]]
Now, we subtract i from each element in each list, where i is 0 for the first list, 1 for the second list, 2 for the third list, etc.

    [[23, 34, 15], [18, 34, 11], [177, 34, 195], [218, 34, 909], [184, 2, 34]]
Then, we take the intersection of all of these lists, which will leave us with one element, 34. This would then indicate that the phrase “the cake is a lie” is in the file in order. And this is true: if we look at the original list, we see that the sequence 34, 35, 36, 37, 38 gives us our phrase.

There’s many more query options you could support (take a look at Google’s advanced search for inspiration). You can try to implement some of these on your own.

Our final step, then, is to implement a query parser that will allow us to combine different types of queries to get a single result set (like how you can type something like ‘the cake “is a lie”’ into Google, which combines standard queries (the entire thing), and phrase queries (“is a lie”). This is very simple to do, as well: just use delimiters (like quotes) to specify a certain type of query, run each smaller query separately, and then intersect all of these result sets to get the final list of documents.

In [15]:
##5) Getting the query from the user
query = raw_input("Enter your query string : ")
query_list = get_list_tokens_string(query)
result_file_indices= []
result = []

start_time = time.time()

##6) Tokenizing query string to get individual words
query_list = get_list_tokens_string(query)

##7) Get the documents including the query terms
for q in query_list:
    if q in inverted_index.keys():
        result_file_indices.append([filename for filename in inverted_index[q.lower()]])
           
setted = set(result_file_indices[0]).intersection(*result_file_indices)

for filename in setted:
    temp = []
    for word in query_list:
        temp.append(inverted_index[word][filename][:])
    for i in range(len(temp)):
        for ind in range(len(temp[i])):
            temp[i][ind] -= i
    if set(temp[0]).intersection(*temp):
        result.append(filename)

print("result" ,result)
##8) Sorting the dictionary based on its values            
result_file_indices = sorted(result_file_dict.items(), key=lambda x:x[1],reverse = True)

if(len(result)==0):
    print "Sorry No matches found"
else:
    print "Number of search results : " , len(result)
    print "Results:"
    print "\n"
    for index in result:
        print get_raw_text(corpus, index)
        print "\n"

Enter your query string : bank crisis
('result', ['test/20259'])
Number of search results :  1
Results:


LOUVRE REAFFIRMATION NOT ENOUGH - U.K. ANALYSTS
  U.S. And West German reaffirmation of
  support for the Louvre Accord cannot cure the fundamental
  problems bedevilling the world economy which lie behind the
  current collapse in stock markets, London economists said.
      "There's going to have to be some acknowledgement that the
  dollar is going to be allowed to slip," said Richard Jeffrey of
  Hoare Govett. "If not, there is going to be continued fear that
  when pressure emerges on the dollar, the Fed will be forced to
  tighten. This throws up the economic abyss of recession in the
  U.S. With obvious knock on effects on the rest of the world."
      But some economists added that Wall Street's crash, which
  dragged other major markets down with it, may help curb the
  very problems that sparked the turmoil - namely world inflation
  fears and the massive and persistent U

# Try some queries